# Duelling Networks

## Problem statement

An evil warlord has discovered an ancient tunnel (represented as a neural network) connected to your city's sewers. He plans to send deadly tidal waves through it to attack your city! Thankfully, you have a plan. You know there is an even older, prehistoric tunnel (also represented as a neural network) that connects to the city sewers at the point just before the warlord's waves reach the city. Let us call this point the mixing chamber.

You are going to duel him for the fate of the city by sending counter waves of your own through the prehistoric tunnel to neutralize the warlords' attacks. Aim to get the energy of the resultant waves (L2 norm of combined neural network output) to be as close to 0 as possible. The warlord will send 5 waves, and his wave patterns are known to you upfront.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
warlord_attacks = [
    [-0.2887,  2.1014, -1.1875, -1.5572, -1.0120, -1.7173,  0.4706,  3.0802, -0.2799, -2.0404,  0.3002, -2.9843],
    [0.0000,  0.0000,  0.0000, -2.9875, -4.0940,  3.0872,  0.0000,  0.0000, 0.0000,  0.0000,  0.0000,  0.0000],
    [0.000,  1.3644,  1.1336, -0.4226, -1.4847, -0.81096,  0.81096,  1.4847,  0.4226, -1.1336, -1.3644,  5.2454e-07],
    [0.8171, -0.7760, -1.2905,  0.5329,  1.1489,  0.1438,  0.8859, -0.3775, -2.2911,  0.7330, -0.9208, -0.4634],
    [0.7770, 0.9937, 0.3484, 1.2489, 1.5510, 1.3883, 1.9447, 1.9054, 2.4041, 2.2864, 2.5607, 3.6302]
]

In [ ]:
class DuellingNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)

        # Warlord's ancient tunnel
        self.warlord = nn.Sequential(
            nn.Linear(12, 32), nn.LeakyReLU(0.01),
            nn.Linear(32, 64), nn.LeakyReLU(0.01),
            nn.Linear(64, 128), nn.LeakyReLU(0.01),
            nn.Linear(128, 256), nn.LeakyReLU(0.01),
            nn.Linear(256, 128),
        )

        # Your prehistoric tunnel
        self.defender = nn.Sequential(
            nn.Linear(12, 32), nn.LeakyReLU(0.01),
            nn.Linear(32, 64), nn.LeakyReLU(0.01),
            nn.Linear(64, 128), nn.LeakyReLU(0.01),
            nn.Linear(128, 256), nn.LeakyReLU(0.01),
            nn.Linear(256, 128),
        )

        # Waves meet in a mixing chamber
        self.mixing_weights = nn.Parameter(torch.randn(128) * 0.1)
        self.mixing_bias = nn.Parameter(torch.randn(128) * 0.1)

        # City gates
        self.city_gates = nn.Sequential(nn.Linear(128, 64), nn.LeakyReLU(0.01), nn.Linear(64, 1))

        # The tunnels are literally set in stone, they are not changing!
        for param in self.parameters():
            param.requires_grad = False

        self.eval();

    def mixing_chamber(self, warlord_force, defender_force):
        interaction = warlord_force * defender_force

        resonance = (
            self.mixing_weights * interaction
            + warlord_force
            + defender_force
            + self.mixing_bias
        )

        return resonance

    def forward(self, attack_wave, counter_wave):
        warlord_force = self.warlord(attack_wave)
        defender_force = self.defender(counter_wave)

        # The forces interact in the mixing chamber
        mixed_waves = self.mixing_chamber(warlord_force, defender_force)

        # The mixed waves crash onto the city gates!!
        destruction = self.city_gates(mixed_waves)

        return destruction

In [ ]:
def assess_defense(
    network: nn.Module,
    attacks: list[list[float]], counter_waves: list[list[float]],
) -> list[float]:
    destructions = []
    energy_limit = 100.0
    tolerance = 0.001
    
    with torch.no_grad():
        for i, (atk, ctr) in enumerate(zip(attacks, counter_waves)):
            atk = torch.FloatTensor(atk)
            ctr = torch.FloatTensor(ctr)

            d = network(atk, ctr).item()
            e = torch.norm(ctr).item()
            ok = abs(d) < tolerance and e <= energy_limit
            print(f"Attack {i+1}: destruction={d:+.5f} counterwave energy={e:.1f}/{energy_limit}  {'OK' if ok else 'FAIL'}")
            destructions.append(d)
    return destructions

If you dont do anything, this is what will happen to the city:

In [ ]:
# All inputs as zero
null_input = [[0] * 12] * 5

In [ ]:
duel = DuellingNetwork()
destruction = assess_defense(duel, warlord_attacks, null_input)

### Your task

Submit a list of 5 counter-wave tensors to match each of the warlord's attacks, each of shape `(12,)`. The prehistoric tunnel is fragile so all of your counter wave input cannot be too big (must have L2 norm ≤ 100.0). Please use the provided `assess_defense` function to check your score before you submit.

### Scoring

Your submission will be evaluated by the average city destruction across all warlord attacks, such that:

- Average destruction of 0.11 (close to inputing all zeros) will grant 0 points
- Average destruction below 0.001 will grant 15 points
- Average destruction between 0.001 and 0.11 will be awarded between 0 to 15 points, scaled logarithmically (i.e. score is higher as destruction approaches 0.001)

Remember that if ANY of your counter waves exceed the wave energy limit, the prehistoric tunnel will crack and you will get 0 points!

You are allowed to make 10 API submissions for this challenge. 

### API submission format

In [ ]:
import requests

In [ ]:
# Copy paste your API key here from the contest webpage, don't show to others!
api_key = ""

In [ ]:
def make_payload(list_of_counterwaves):
    return {"solution": list_of_counterwaves}

In [ ]:
def post_answer(payload: dict):
    url = "https://competitions.aiolympiad.my/api/maio2026_qualis/maio2026_duelling_networks"
    response = requests.post(url=url, json=payload, headers={"X-API-Key": api_key})
    if response.status_code == 200:
        return response.json()
    else:
        return (
            f"Failed to submit, status code is {response.status_code}\n{response.text}"
        )

Example API submission:

```python
>>> null_input = [[0] * 12] * 5
>>> payload = make_payload(null_input)
>>> post_answer(payload=payload)
{'status': 'SUCCESS',
 'message': 'Answer for challenge maio2026_duelling_networks submitted successfully on 2026-02-20 00:00:00.00+00:00. Total submissions is 1 / 10.'}
```

## Your work below!

In [ ]:
def assess_defense(
    network: nn.Module,
    attacks: list[list[float]], counter_waves: list[list[float]],
) -> list[float]:
    destructions = []
    energy_limit = 100.0
    tolerance = 0.001
    
    with torch.no_grad():
        for i, (atk, ctr) in enumerate(zip(attacks, counter_waves)):
            atk = torch.FloatTensor(atk)
            ctr = torch.FloatTensor(ctr)

            d = network(atk, ctr).item()
            e = torch.norm(ctr).item()
            ok = abs(d) < tolerance and e <= energy_limit
            print(f"Attack {i+1}: destruction={d:+.5f} counterwave energy={e:.1f}/{energy_limit}  {'OK' if ok else 'FAIL'}")
            destructions.append(d)
    return destructions

def find_counter_waves(network, attacks, energy_limit=100.0, steps=3000, lr=0.01):
    network.eval()

    # Convert attacks to tensor
    atk_tensor = torch.FloatTensor(attacks)

    # Counter waves are the ONLY trainable parameters
    counter_waves = torch.zeros_like(atk_tensor, requires_grad=True)

    optimizer = optim.Adam([counter_waves], lr=lr)

    for step in range(steps):
        optimizer.zero_grad()

        destruction = network(atk_tensor, counter_waves)

        # Minimise squared destruction (L2 norm)
        loss = torch.mean(destruction ** 2)

        loss.backward()
        optimizer.step()

        # ---- Enforce energy constraint (projection onto L2 ball) ----
        with torch.no_grad():
            norms = torch.norm(counter_waves, dim=1, keepdim=True)
            scale = torch.clamp(energy_limit / (norms + 1e-8), max=1.0)
            counter_waves *= scale

        if step % 500 == 0:
            print(f"Step {step}: loss={loss.item():.8f}")

    return counter_waves.detach()

duel = DuellingNetwork()

optimized_counters = find_counter_waves(
    duel,
    warlord_attacks,
    energy_limit=100.0,
    steps=4000,
    lr=0.01,
)

destruction = assess_defense(
    duel,
    warlord_attacks,
    optimized_counters.tolist()
)

print("\nOptimized Counter Waves:\n")
print(optimized_counters)

